In [1]:
import openrouteservice as ors
import folium
from pandas import DataFrame
import pandas as pd
import xlrd

In [2]:
rent = pd.read_csv("/home/ads/generic-real-estate-consulting-project-group-55/rent_data/renew(15)", index_col=0)
train = pd.read_excel("/home/ads/generic-real-estate-consulting-project-group-55/data/raw/train_station_20to21.xlsx", index_col=0)
distance_duration = pd.read_csv("/home/ads/generic-real-estate-consulting-project-group-55/data/raw/distances_durations(19)", index_col=0).reset_index()
client = ors.Client(key='5b3ce3597851110001cf6248d3629fc815a24f0582ed2410062c812f')

##### function: Draw a map based on the data obtained by comparing the house with the station
##### para: map

In [4]:
def coordinates(rent, train, distance_duration):
    get_data = distance_duration.loc[0]
    coordinates = [[float(rent[rent['listingSlug'] ==  get_data['rent_address']]['longitude']), float(rent[rent['listingSlug'] ==  get_data['rent_address']]['latitude'])], \
    [float(train[train['Stop_name'] == get_data['stop_name']]['Stop_long']), float(train[train['Stop_name'] == get_data['stop_name']]['Stop_lat'])]]
    #When drawing, first latitude and then longitude, when calculating distance, first longitude and then latitude
    coordinates2 = [[float(rent[rent['listingSlug'] ==  get_data['rent_address']]['latitude']), float(rent[rent['listingSlug'] ==  get_data['rent_address']]['longitude'])], \
    [float(train[train['Stop_name'] == get_data['stop_name']]['Stop_lat']), float(train[train['Stop_name'] == get_data['stop_name']]['Stop_long'])]]
    map = folium.Map(location=coordinates2[0], tiles='cartodbpositron', zoom_start=13)
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
        )
    folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]).add_to(map)
    folium.Marker(
    location=coordinates2[0],
    popup="Rent",
    icon=folium.Icon(color="green"),
    ).add_to(map)
    folium.Marker(
    location=coordinates2[1],
    popup="train",
    icon=folium.Icon(color="red", icon="info-sign"),
    ).add_to(map)
    return map
    


In [5]:
coordinates(rent, train, distance_duration)

In [6]:
coordinates(rent, train, distance_duration).save('/home/ads/generic-real-estate-consulting-project-group-55/data/raw/compare_map.html')